In [4]:
import pandas as pd

In [5]:
df = pd.read_csv('/Users/shreyu/VSCODE/junk/UoN-Business-Analytics/FBA/coursework/cwk_data_20811152.csv')

In [6]:
df.describe()

,age,balance,day,duration,campaign,pdays,previous
count,4000.0000,4000.00000,4000.00000,4000.000000,4000.000000,4000.000000,4000.000000
mean,41.0355,1344.92125,15.93175,293.145000,2.691500,45.795500,0.651500
std,10.7738,2662.04662,8.32742,302.032846,3.026321,104.807544,1.909309
min,18.0000,-1884.00000,1.00000,6.000000,1.000000,-1.000000,0.000000
25%,33.0000,75.00000,8.00000,112.000000,1.000000,-1.000000,0.000000
50%,39.0000,459.00000,16.00000,200.000000,2.000000,-1.000000,0.000000
75%,48.0000,1524.50000,21.00000,357.000000,3.000000,-1.000000,0.000000
max,95.0000,39098.00000,31.00000,4918.000000,33.000000,854.000000,27.000000


In [8]:
df_copy_1 = df.copy()
df_copy_1

,age,job,marital,education,default,balance,housing,loan,contact,day,duration,campaign,pdays,previous,poutcome,y
0,40,management,married,tertiary,no,-17,yes,yes,cellular,11,474,1,256,1,success,yes
1,55,management,divorced,tertiary,no,2037,no,no,cellular,4,305,2,94,3,success,yes
2,34,student,single,unknown,no,1011,no,no,cellular,1,253,1,-1,0,unknown,yes
3,39,technician,married,secondary,no,4101,yes,no,cellular,18,146,2,264,1,other,yes
4,26,blue-collar,single,secondary,no,1006,yes,no,cellular,14,667,1,-1,0,unknown,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,31,blue-collar,married,primary,no,26,yes,no,unknown,20,51,26,-1,0,unknown,no
3996,46,management,single,tertiary,no,2154,yes,no,cellular,21,87,4,-1,0,unknown,no
3997,32,management,married,tertiary,no,132,yes,no,unknown,30,565,2,-1,0,unknown,no
3998,53,management,married,tertiary,no,2578,no,no,cellular,19,299,4,107,2,failure,no


In [12]:
df_copy_1.isnull().sum()

age          0
job          0
marital      0
education    0
default      0
balance      0
housing      0
loan         0
contact      0
day          0
duration     0
campaign     0
pdays        0
previous     0
poutcome     0
y            0
dtype: int64

In [13]:
df_copy_1.notnull().sum()

age          4000
job          4000
marital      4000
education    4000
default      4000
balance      4000
housing      4000
loan         4000
contact      4000
day          4000
duration     4000
campaign     4000
pdays        4000
previous     4000
poutcome     4000
y            4000
dtype: int64